In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

2025-03-12 15:40:32.719132: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-12 15:40:33.986765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-12 15:40:33.986823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-12 15:40:34.179322: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 15:40:34.493038: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Step 1: Generate random data (10 features, 1 output)
n_samples = 1000
n_features = 10

In [ ]:
# Generate random input features (X) and output (y)
X = np.random.randn(n_samples, n_features)
y = 2 * np.sum(X, axis=1) + np.random.randn(n_samples)  # Simple linear relation + noise

# Split the data into training, validation, and test sets (80% train, 10% validation, 10% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Step 2: Define a simple fully connected neural network for Phase I (bootstrap models)
def create_bootstrap_nn(input_shape):
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Step 3: Define the noise variance estimation network (Phase II)
def create_noise_variance_nn(input_shape):
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)  # Output layer for variance prediction
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Step 4: Train bootstrap neural networks (Phase I)
def train_bootstrap_nns(X_train, y_train, B=10):
    bootstrap_models = []
    bootstrap_predictions = []

    # Generate B bootstrap samples and train models
    for _ in range(B):
        # Create a bootstrap sample by sampling with replacement
        indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_bootstrap = X_train[indices]
        y_bootstrap = y_train[indices]

        # Create and train a new model
        model = create_bootstrap_nn(X_train.shape[1:])
        model.fit(X_bootstrap, y_bootstrap, epochs=50, batch_size=32, verbose=0)
        bootstrap_models.append(model)

        # Store predictions on the original training data
        predictions = model.predict(X_train)
        bootstrap_predictions.append(predictions)

    return bootstrap_models, np.array(bootstrap_predictions)

# Step 5: Custom likelihood-based cost function (Equation 12)
def custom_loss(y_true, y_pred, model_variance):
    # r^2(x_i) is already the residual variance computed earlier
    r_squared = compute_r_squared(y_true, y_pred, model_variance)
    
    # Compute the log of the predicted variance
    term_1 = tf.math.log(y_pred + 1e-10)  # Add a small epsilon for numerical stability
    
    # Compute the ratio term
    term_2 = r_squared / y_pred
    
    # Combine both terms for the likelihood-based loss function
    loss = 0.5 * tf.reduce_mean(tf.square(term_1 + term_2))
    return loss

# Compute r^2(x_i) for each bootstrap model
def compute_r_squared(y_true, y_pred, model_variance):
    residuals = (y_true - y_pred) ** 2 - model_variance
    return np.maximum(residuals, 0)

# Step 6: Phase II - Train the noise variance estimation network (NNₑ)
def train_noise_variance_nn_custom_loss(X_train, y_train, bootstrap_predictions, B=10):
    # Calculate r^2(x_i) for each bootstrap model
    residuals_all = []
    for b in range(B):
        residuals = compute_r_squared(y_train, bootstrap_predictions[b], np.var(bootstrap_predictions, axis=0))
        residuals_all.append(residuals)
    
    residuals_all = np.stack(residuals_all, axis=-1)
    mean_residuals = np.mean(residuals_all, axis=-1)  # Averaging residuals across bootstrap models

    # Create and train the NNₑ for noise variance estimation using the custom loss function
    nn_e = create_noise_variance_nn(X_train.shape[1:])
    
    # Use the custom loss function for training
    model_variance = np.var(bootstrap_predictions, axis=0)  # Variance across bootstrap predictions
    nn_e.compile(optimizer='adam', loss=lambda y_true, y_pred: custom_loss(y_true, y_pred, model_variance))
    
    nn_e.fit(X_train, mean_residuals, epochs=50, batch_size=32)
    
    return nn_e

# Step 7: Train the models
# Train the bootstrap models (Phase I)
bootstrap_models, bootstrap_predictions = train_bootstrap_nns(X_train, y_train, B=10)

# Train the noise variance estimation model (Phase II) using the custom likelihood-based loss function
nn_e_custom = train_noise_variance_nn_custom_loss(X_train, y_train, bootstrap_predictions, B=10)

# Step 8: Evaluate the models on the test set
# Make predictions with the bootstrap models
bootstrap_preds_test = np.array([model.predict(X_test) for model in bootstrap_models])

# Calculate the mean prediction across all bootstrap models
bootstrap_mean_preds_test = np.mean(bootstrap_preds_test, axis=0)

# Estimate the noise variance using NNₑ
predicted_variance_test = nn_e_custom.predict(X_test)

# Print results
print(f"Mean prediction for test set: {np.mean(bootstrap_mean_preds_test)}")
print(f"Estimated noise variance (NNₑ) for test set: {np.mean(predicted_variance_test)}")

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Step 1: Generate random data
n_samples = 1000
n_features = 10

# Generate random input features (X) and output (y)
X = np.random.randn(n_samples, n_features)
y = 2 * np.sum(X, axis=1) + np.random.randn(n_samples)  # Simple linear relation + noise

# Split the data into training, validation, and test sets (80% train, 10% validation, 10% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Step 2: Define a simple fully connected neural network for Phase I (bootstrap models)
def create_bootstrap_nn(input_shape):
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Step 3: Define the noise variance estimation network (Phase II)
def create_noise_variance_nn(input_shape):
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)  # Output layer for variance prediction
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Step 4: Train bootstrap neural networks (Phase I)
def train_bootstrap_nns(X_train, y_train, B=10):
    bootstrap_models = []
    bootstrap_predictions = []

    # Generate B bootstrap samples and train models
    for _ in range(B):
        # Create a bootstrap sample by sampling with replacement
        indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_bootstrap = X_train[indices]
        y_bootstrap = y_train[indices]

        # Create and train a new model
        model = create_bootstrap_nn(X_train.shape[1:])
        model.fit(X_bootstrap, y_bootstrap, epochs=50, batch_size=32, verbose=0)
        bootstrap_models.append(model)

        # Store predictions on the original training data
        predictions = model.predict(X_train)
        bootstrap_predictions.append(predictions)

    return bootstrap_models, np.array(bootstrap_predictions)

# Custom loss function for likelihood-based cost function (Equation 12)
def custom_loss(y_true, y_pred, model_variance):
    # r^2(x_i) is already the residual variance computed earlier
    residuals = (y_true - y_pred) ** 2 - model_variance
    r_squared = tf.maximum(residuals, 0)  # Ensure non-negative residuals
    
    # Compute the log of the predicted variance (using TensorFlow log)
    term_1 = tf.math.log(y_pred + 1e-10)  # Add a small epsilon for numerical stability
    
    # Compute the ratio term
    term_2 = r_squared / y_pred
    
    # Combine both terms for the likelihood-based loss function
    loss = 0.5 * tf.reduce_mean(tf.square(term_1 + term_2))
    return loss

# Compute r^2(x_i) for each bootstrap model
def compute_r_squared(y_true, y_pred, model_variance):
    residuals = (y_true - y_pred) ** 2 - model_variance
    return np.maximum(residuals, 0)

# Step 6: Phase II - Train the noise variance estimation network (NNₑ)
def train_noise_variance_nn_custom_loss(X_train, y_train, bootstrap_predictions, B=10):
    # Calculate r^2(x_i) for each bootstrap model
    residuals_all = []
    for b in range(B):
        residuals = compute_r_squared(y_train, bootstrap_predictions[b], np.var(bootstrap_predictions, axis=0))
        residuals_all.append(residuals)
    
    residuals_all = np.stack(residuals_all, axis=-1)
    mean_residuals = np.mean(residuals_all, axis=-1)  # Averaging residuals across bootstrap models

    # Create and train the NNₑ for noise variance estimation using the custom loss function
    nn_e = create_noise_variance_nn(X_train.shape[1:])
    
    # Use the custom loss function for training
    model_variance = np.var(bootstrap_predictions, axis=0)  # Variance across bootstrap predictions
    nn_e.compile(optimizer='adam', loss=custom_loss(y_true, y_pred, model_variance))
    
    nn_e.fit(X_train, mean_residuals, epochs=50, batch_size=32)
    
    return nn_e

# Step 7: Train the models
# Train the bootstrap models (Phase I)
bootstrap_models, bootstrap_predictions = train_bootstrap_nns(X_train, y_train, B=10)

# Train the noise variance estimation model (Phase II) using the custom likelihood-based loss function
nn_e_custom = train_noise_variance_nn_custom_loss(X_train, y_train, bootstrap_predictions, B=10)

# Step 8: Evaluate the models on the test set
# Make predictions with the bootstrap models
bootstrap_preds_test = np.array([model.predict(X_test) for model in bootstrap_models])

# Calculate the mean prediction across all bootstrap models
bootstrap_mean_preds_test = np.mean(bootstrap_preds_test, axis=0)

# Estimate the noise variance using NNₑ
predicted_variance_test = nn_e_custom.predict(X_test)



25/25 [==============================] - 0s 1ms/step


NameError: name 'y_true' is not defined